#qgrid - SlickGrid in IPython notebook
---

Qgrid is an IPython extension which uses a javascript library called SlickGrid to render pandas DataFrames within an IPython notebook.  It's being developed for use in [Quantopian's hosted research environment]( https://www.quantopian.com/research?utm_source=github&utm_medium=web&utm_campaign=qgrid-nbviewer), and this notebook demonstrates the current state of the project.

## Overview
* [SlickGrid](https://github.com/mleibman/SlickGrid) is an an advanced javascript grid which allows users to scroll, sort, 
and filter hundreds of thousands of rows with extreme responsiveness.  
* [Pandas](https://github.com/pydata/pandas) is a powerful data analysis / manipulation library for Python, and DataFrames are the primary way of storing and manipulating two-dimensional data in pandas.

[Qgrid](https://github.com/quantopian/qgrid) renders pandas DataFrames as SlickGrids, which enables users to explore the entire contents of a DataFrame using intuitive sorting and filtering controls.  It's designed to be used within IPython notebook, and it's also fully functional when rendered in [nbviewer](http://nbviewer.ipython
.org/github/quantopian/qgrid/blob/master/qgrid_demo.ipynb).

## Installation using 'pip install'
##### Qgrid is a python module so you can install it using pip:

```pip install git+https://github.com/quantopian/qgrid```

##### Import it into your namespace like you would for any other python module:

In [3]:
import qgrid
qgrid.load_js()

qgrid loaded

In [4]:
import pandas as pd
import numpy as np
randn = np.random.randn

pd.set_option('display.max_rows', 8)

from pandas.io.data import get_data_yahoo
spy = get_data_yahoo(
    symbols='SPY',
    start=pd.Timestamp('2011-01-01'),
    end=pd.Timestamp('2014-01-01'),
    adjust_price=True,
)
qgrid.SlickGrid(data_frame=spy, remote_js=False)

##### Prepare non-python dependencies by calling 'nbinstall':

In [22]:
qgrid.nbinstall()  # copies javascript dependencies to your /nbextensions folder 

## Demo 1 - Rendering a DataFrame returned by Yahoo Finance
##### First, lets create a sample DataFrame using pandas 'get_data_yahoo' function:

In [3]:
import pandas as pd
import numpy as np
randn = np.random.randn

pd.set_option('display.max_rows', 8)

from pandas.io.data import get_data_yahoo
spy = get_data_yahoo(
    symbols='SPY',
    start=pd.Timestamp('2011-01-01'),
    end=pd.Timestamp('2014-01-01'),
    adjust_price=True,
)

##### BEFORE - Here's IPython's default representation of our 'spy' DataFrame:

In [18]:
spy

,Open,High,Low,Close,Volume,Adj_Ratio
Date,,,,,,
2011-01-03,115.648405,116.460708,114.726575,115.958727,138725200,0.912701
2011-01-04,116.214283,116.250792,115.173803,115.894838,137409700,0.912701
2011-01-05,115.529756,116.570235,115.420229,116.497217,133975300,0.912701
2011-01-06,116.542855,116.670633,115.922218,116.269042,122519000,0.912701
...,...,...,...,...,...,...
2013-12-26,178.160131,178.762625,178.140706,178.665444,63365000,0.971747
2013-12-27,178.898669,178.976396,178.471098,178.655732,61814000,0.971747
2013-12-30,178.675157,178.820928,178.393357,178.626581,56857000,0.971747
2013-12-31,178.869518,179.471996,178.733459,179.471996,86119900,0.971747


##### AFTER - Here's the same 'spy' DataFrame being rendered as a qgrid:

In [5]:
qgrid.SlickGrid(data_frame=spy, remote_js=False)

KeyError: 'precision'

In [25]:
qgrid.set_defaults(remote_js=False)

In [13]:
qgrid.set_js_option('forceFitColumns', True)

In [14]:
qgrid.show_grid(spy, remote_js=False)

## Demo 2 - Rendering a sample DataFrame that includes a MultiIndex
##### Now let's create another DataFrame, but let's give it a MultiIndex this time:

In [6]:
tuples = zip(*[['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],
          ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']])


index = pd.MultiIndex.from_tuples(tuples, names=['first', 'second'])
multi_index_df = pd.DataFrame(randn(8, 2), index=index, columns=['A', 'B'])

##### BEFORE - Here's IPython's default representation of our 'multi_index_df' DataFrame:

In [7]:
multi_index_df

A         B
first second                    
bar   one     1.502425  0.530116
      two    -0.749106 -0.585371
baz   one    -0.787861 -0.333152
      two     1.821110  0.892432
foo   one    -1.154201 -0.073072
      two     0.348630 -0.174053
qux   one    -0.914489 -1.485095
      two    -0.629482 -1.601456

##### AFTER - Here's the same 'multi_index_df' DataFrame being rendered as a qgrid:

In [8]:
qgrid.show_grid(multi_index_df, remote_js=True)

## API & Usage
### qgrid.show_grid (data_frame, remote_js=False)
This is the most interesting function provided by the qgrid module.  When run within an IPython notebook, it takes a pandas DataFrame and outputs a SlickGrid.

##### Parameters:
  1. **data_frame** - The pandas.DataFrame object to render as a SlickGrid.
  2. **remote_js** - A boolean value indicating whether should load javascript from the local '/nbextensions' folder or directly from a CDN.  Defaults to False.
    * The ability to load javascript dependencies from remote sources is what allows qgrids to be fully functional when viewing notebooks using nbviewer.  In order for a particular qgrid to render properly in nbviewer, it must be generated while remote_js is set to true.
    * When there are multiple calls to show_grid in a notebook, **ONLY the first call to show_grid has any effect on the value of remote_js for the lifetime of the page**.  This is because once the dependencies are loaded into the browser, they aren't unloaded until the user navigates away or refreshes the page.

### qgrid.nbinstall (overwrite=False)
This function is used during installation to prepare qgrid's non-python dependencies.  The qgrid module contains a 'qgridjs' folder that holds a bunch of javascript/css files, and this function creates a copy of 'qgridjs' in the "/nbextensions" folder for your IPython installation.
##### Parameters:
1. **overwrite** - A boolean value indicating whether installation should continue when the 'qgridjs' folder already exists in '/nbextensions'.  Defaults to True.

## Running from source using 'git clone'
##### Use git to clone the qgrid repository to your hard drive
`git clone git@github.com:quantopian/qgrid.git`
##### Set the root folder of your qgrid repository to be the working directory for your IPython notebook server
This is an easy way to get qgrid to run directly from the source code.  IPython notebook will look in it's working directory for modules to load, so qgrid will immediately become available for importing.

`ipython notebook --notebook-dir=~/path/to/qgrid/repo`

It's not always convenient to use the qgrid repository as your working directory for IPython noteboook, so I often create symbolic-links from qgrid's source code into the "/nbextensions" and "/extensions" folders under my IPython directory.


##### Import it into your namespace like you would for any other python module:

In [ ]:
import qgrid

##### Prepare non-python dependencies by calling 'nbinstall':

In [ ]:
qgrid.nbinstall(overwrite=True)  # use overwrite=True to keep your /nbextensions folder up to date during development